In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

from custom_estimator import Estimator
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import os
import glob
from sklearn.externals import joblib
from encoding import FreqeuncyEncoding
import warnings
warnings.filterwarnings("ignore")
from sklearn.externals import joblib

In [ ]:
train=pd.read_pickle('../train_v2.pkl')
test=pd.read_pickle('../test_v2.pkl')

In [ ]:
feat1=pd.read_csv('../feats_user_coup_item_no_lag_v2.csv')
feat2=pd.read_csv('../coupon_item_feats_no_lag_v2.csv')
feat3=pd.read_csv('../brand_cust_feats_no_lag_v2.csv')
feat4=pd.read_csv('../brand_feats_no_lag_v2.csv')
feat6=pd.read_csv('../coupon_mean_encoding_v2.csv')
feat7=pd.read_csv('../camp_cust_feats_v2.csv')
feat8=pd.read_csv('../cust_coup_hist3_v2.csv')
feat9=pd.read_csv('../customer_coupon_var1_v2.csv')
feat10=pd.read_csv('../customer_coupon_var2_v2.csv')
feat11=pd.read_csv('../coup_hist1_v2.csv')
feat12=pd.read_csv('../test_feat_2.csv')
feat13=pd.read_csv('../test_feat.csv')

In [ ]:
drop_cols=['coupon_discount_mean', 'selling_price_mean', 'other_discount_sum', 'other_discount_mean', 'cd_sp_ratio_mean', 
 'cd_sp_ratio_sum', 'coupon_discount_mean_coup', 'selling_price_mean_coup', 'other_discount_sum_coup',
 'other_discount_mean_coup', 'cd_sp_ratio_mean_coup', 'cd_sp_ratio_sum_coup', 'selling_price_sum_brand_cust',
 'coupon_discount_mean_brand_cust', 'selling_price_mean_brand_cust', 'other_discount_sum_brand_cust', 
 'other_discount_mean_brand_cust', 'cd_sp_ratio_mean_brand_cust', 'cd_sp_ratio_sum_brand_cust', 'selling_price_sum_brand', 
 'coupon_discount_mean_brand', 'selling_price_mean_brand', 'other_discount_sum_brand', 'other_discount_mean_brand', 
 'cd_sp_ratio_mean_brand', 'cd_sp_ratio_sum_brand', 'selling_price_sum_camp_cust', 'coupon_discount_mean_camp_cust', 
 'selling_price_mean_camp_cust', 'other_discount_sum_camp_cust', 'other_discount_mean_camp_cust',
 'cd_sp_ratio_mean_camp_cust', 'cd_sp_ratio_sum_camp_cust', 'selling_price_sum_cust_var3', 'coupon_discount_mean_cust_var3',
 'selling_price_mean_cust_var3', 'other_discount_sum_cust_var3', 'other_discount_mean_cust_var3', 
 'cd_sp_ratio_mean_cust_var3', 'cd_sp_ratio_sum_cust_var3', 'selling_price_sum_cust_var1', 'coupon_discount_mean_cust_var1',
 'selling_price_mean_cust_var1', 'other_discount_sum_cust_var1', 'other_discount_mean_cust_var1', 'cd_sp_ratio_mean_cust_var1',
 'cd_sp_ratio_sum_cust_var1', 'selling_price_sum_cust_var2', 'coupon_discount_mean_cust_var2', 'selling_price_mean_cust_var2',
 'other_discount_sum_cust_var2', 'other_discount_mean_cust_var2', 'cd_sp_ratio_mean_cust_var2', 'cd_sp_ratio_sum_cust_var2', 
 'selling_price_sum_coup_var1', 'coupon_discount_mean_coup_var1', 'selling_price_mean_coup_var1', 
 'other_discount_sum_coup_var1', 'other_discount_mean_coup_var1', 'cd_sp_ratio_mean_coup_var1', 'cd_sp_ratio_sum_coup_var1']
len(drop_cols)

In [ ]:
train=train.merge(feat1,on='id')
test=test.merge(feat1,on='id')
train=train.merge(feat2,on='id')
test=test.merge(feat2,on='id')
train=train.merge(feat3,on='id')
test=test.merge(feat3,on='id')
train=train.merge(feat4,on='id')
test=test.merge(feat4,on='id')
train=train.merge(feat6,on='coupon_id')
test=test.merge(feat6,on='coupon_id')
train=train.merge(feat7,on=['campaign_id','customer_id'])
test=test.merge(feat7,on=['campaign_id','customer_id'])
train=train.merge(feat8,on=['campaign_id','customer_id'])
test=test.merge(feat8,on=['campaign_id','customer_id'])
train=train.merge(feat9,on=['campaign_id','customer_id'])
test=test.merge(feat9,on=['campaign_id','customer_id'])
train=train.merge(feat10,on=['campaign_id','customer_id'])
test=test.merge(feat10,on=['campaign_id','customer_id'])
train=train.merge(feat11,on=['campaign_id','coupon_id'])
test=test.merge(feat11,on=['campaign_id','coupon_id'])
train=train.merge(feat12,on='id')
test=test.merge(feat12,on='id')
train=train.merge(feat13,on='id')
test=test.merge(feat13,on='id')


In [ ]:
train.drop(drop_cols,axis=1,inplace=True)
test.drop(drop_cols,axis=1,inplace=True)

In [ ]:
train.shape

In [ ]:
target=train[['id','redemption_status']]
train.drop('redemption_status',axis=1,inplace=True)
df=pd.concat((train,test),axis=0)

In [ ]:
df['cat1']=(df['customer_id'].astype('str')+'_'+df['campaign_id'].astype('str'))

In [ ]:
df.shape

In [ ]:
df['num_coupons_per_camp']=df['campaign_id'].map(df.groupby('campaign_id')['coupon_id'].nunique().to_dict())
df['num_cust_per_camp']=df['campaign_id'].map(df.groupby('campaign_id')['customer_id'].nunique().to_dict())
df['num_cust_per_coup']=df['coupon_id'].map(df.groupby('coupon_id')['customer_id'].nunique().to_dict())


In [ ]:
df.head()

In [ ]:
cat_cols=['campaign_id','coupon_id','customer_id','campaign_type','campaign_start_day','campaign_start_dow',
         'campaign_end_day','campaign_end_dow','item_id',
          'age_range','marital_status','family_size','no_of_children',
          'cat1'
         ]
id_cols=['id']
date_cols=['start_date','end_date']

In [ ]:
from sklearn.model_selection import GroupKFold
folds=GroupKFold(n_splits=5)
temp=df[df.id.isin(train.id)]
folds1=[(x,y) for x,y in  folds.split(temp,temp.merge(target,on='id').redemption_status,temp.coupon_id)]
folds2=[(x,y) for x,y in  folds.split(temp,temp.merge(target,on='id').redemption_status,temp.campaign_id)]

In [ ]:
fe=FreqeuncyEncoding(normalize=1,return_df=True,categorical_columns=cat_cols)
df=fe.fit_transform(df)

In [ ]:
use_cols=df.columns[~df.columns.isin(date_cols+id_cols)].tolist()

In [ ]:
train_df=df[df.id.isin(train.id)]
test_df=df[df.id.isin(test.id)]
train_df.shape,test_df.shape

In [ ]:
train_df=train_df.merge(target,on='id')

In [ ]:
params2={'boosting_type': 'gbdt',
 'colsample_bytree': 0.7000000000000001,
 'learning_rate': 0.1,
 'metric': 'None',
 'min_child_weight': 30.0,
 'n_estimators': 5000,
 'n_jobs': -1,
 'num_leaves': 64,
 'objective': 'binary',
 'subsample': 0.8,
 'subsample_freq': 5}
est_lgb=Estimator(model=LGBMClassifier(**params2),n_jobs=-1,early_stopping_rounds=300,
              validation_scheme=folds2
                 )
xgb_params={'colsample_bytree': 0.4,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 30.0,
 'n_estimators': 5000,
 'objective': 'binary:logistic',
 'subsample': 1.0}
est_xgb=Estimator(model=XGBClassifier(**xgb_params),n_jobs=-1,early_stopping_rounds=100,
              validation_scheme=folds2
                 )


In [ ]:
oof_lgb=est_lgb.fit_transform(train_df[use_cols].values,train_df['redemption_status'].values)
est_lgb.avg_cv_score

In [ ]:
est_lgb.feature_importance_df(train_df[use_cols].columns)

In [ ]:
oof_xgb=est_xgb.fit_transform(train_df[use_cols].values,train_df['redemption_status'].values)
est_xgb.avg_cv_score

In [ ]:
est_xgb.feature_importance_df(train_df[use_cols].columns)

In [ ]:
sub=pd.read_csv('../sample_submission_Byiv0dS.csv')

In [ ]:
preds_lgb=est_lgb.transform(test_df[use_cols].values)
preds_xgb=est_xgb.transform(test_df[use_cols].values)

In [ ]:
pd.DataFrame({'id':train_df['id'],'redemption_status': oof_xgb}).to_csv('OOF_baseline_xgb_grp_k_camp_more_feats.csv',index=False)
pd.DataFrame({'id':train_df['id'],'redemption_status': oof_lgb}).to_csv('OOF_baseline_lgb_grp_k_camp_more_feats.csv',index=False)

In [ ]:
pd.DataFrame({'id':test_df['id'],'redemption_status': preds_xgb}).to_csv('baseline_xgb_grp_k_camp_more_feats.csv',index=False)
pd.DataFrame({'id':test_df['id'],'redemption_status': preds_lgb}).to_csv('baseline_lgb_grp_k_camp_more_feats.csv',index=False)